# Sentiment analysis of financial data using Hugging face LLM models
This notebook aims to provide an introduction to documenting an LLM model using the ValidMind Developer Framework. The use case presented is a sentiment analysis of financial pharse data (https://huggingface.co/datasets/financial_phrasebank).

- Initializing the ValidMind Developer Framework
- Running a test various tests to quickly generate document about the data and model

## Before you begin

To use the ValidMind Developer Framework with a Jupyter notebook, you need to install and initialize the client library first, along with getting your Python environment ready.

If you don't already have one, you should also [create a documentation project](https://docs.validmind.ai/guide/create-your-first-documentation-project.html) on the ValidMind platform. You will use this project to upload your documentation and test results.

## Install the client library


In [1]:
# %pip install --upgrade validmind

## Initialize the client library

In a browser, go to the **Client Integration** page of your documentation project and click **Copy to clipboard** next to the code snippet. This code snippet gives you the API key, API secret, and project identifier to link your notebook to your documentation project.

::: {.column-margin}
::: {.callout-tip}
This step requires a documentation project. [Learn how you can create one](https://docs.validmind.ai/guide/create-your-first-documentation-project.html).
:::
:::

Next, replace this placeholder with your own code snippet:

In [2]:
## Replace the code below with the code snippet from your project ## 

import validmind as vm
  
vm.init(
    api_host = "https://api.prod.validmind.ai/api/v1/tracking",
    api_key = "...",
    api_secret = "...",
    project = "..."
)

2023-09-08 16:59:14,966 - INFO(validmind.api_client): Connected to ValidMind. Project: Sentiment Analysis GPT - Initial Validation (cllmzt1d000bhue8h9ibsrybh)


### Load Dataset

In this section, we'll load the financial phrasebank dataset, which will be the foundation for our sentiment analysis tasks.

In [3]:
import pandas as pd

df = pd.read_csv('./datasets/sentiments.csv')
sample = df.sample(10)
sample

,Sentiment,Sentence
4501,positive,The company said it estimates to make a slight...
1591,neutral,Russia wants to utilise its huge forest reserv...
52,positive,"Within the framework of the partnership , Noki..."
2706,neutral,Work on the assignment has already started and...
4566,negative,Dealers said the share was largely hit by inve...
3621,neutral,"In addition , the company is considering the s..."
793,positive,"When the situation normalises , the company wi..."
2336,neutral,"BasWare is headquartered in Espoo , Finland ."
3211,neutral,The company will propose a dividend of EUR0 .1...
4527,neutral,Rosen was cautious about being too optimistic ...


## NLP data quality tests

Before we proceed with the analysis, it's crucial to ensure the quality of our NLP data. This section runs a data quality test plan to validate the data's integrity and suitability.

In [4]:
vm_ds = vm.init_dataset(dataset=df, type="generic", text_column='Sentence', target_column="Sentiment")
text_data_test_plan = vm.run_test_plan("text_data_quality",
                                       dataset=vm_ds)

2023-09-08 16:59:15,014 - INFO(validmind.client): The 'type' argument to init_dataset() argument is deprecated and no longer required.
2023-09-08 16:59:15,014 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/andres/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/andres/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Hugging face transformers

### Example model wrapper

The following code template showcases how to wrap a Hugging Face model for compatibility with the ValidMind Developer Framework. We will load an example model using the transformers API and then run some predictions on our test dataset.

The ValidMind developer framework provides support for Hugging Face transformers out of the box, so in the following section we will show how to initialize multiple transformers models with the `init_model` function, removing the need for a custom wrapper. In cases where you need extra pre-processing or post-processing steps, you can use the following code template as a starting point to wrap your model.

In [5]:
from transformers import pipeline
from dataclasses import dataclass
@dataclass

class Sentiment_HuggingFace:
    """
    A VM Model instance wrapper only requires a predict and predict_proba method.
    """

    predicted_prob_values = None

    def __init__(self, pipeline_task, model_name=None, model=None, tokenizer=None):
        self.model_name = model_name
        self.pipeline_task = pipeline_task
        self.model = pipeline(pipeline_task, model=model, tokenizer=tokenizer)

    def predict(self, data):
        data = [str(datapoint) for datapoint in data]
        results = []
        results = self.model(data)
        results_df = pd.DataFrame(results)
        self.predicted_prob_values = results_df.score.values
        return results_df.label.values

    def predict_proba(self):
        if self.predicted_prob_values is None:
            raise ValueError("First run predict method to retrieve predicted probabilities")
        return self.predicted_prob_values


In [6]:
from transformers import BertTokenizer, BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained("ahmedrachid/FinancialBERT-Sentiment-Analysis",num_labels=3)
tokenizer = BertTokenizer.from_pretrained("ahmedrachid/FinancialBERT-Sentiment-Analysis")
sentiment_model_hf = Sentiment_HuggingFace("sentiment-analysis", model=model, tokenizer=tokenizer)

df_test = df.head(15)

y_pred = sentiment_model_hf.predict(df_test.Sentence.values.tolist())
y_pred_prob = sentiment_model_hf.predict_proba()
df_results = df_test.copy()
df_results['y_pred'] = y_pred
df_results['y_pred_prob'] = y_pred_prob
df_results.head(10)

,Sentiment,Sentence,y_pred,y_pred_prob
0,neutral,"According to Gran , the company has no plans t...",neutral,0.988819
1,neutral,Technopolis plans to develop in stages an area...,neutral,0.999853
2,negative,The international electronic industry company ...,negative,0.999682
3,positive,With the new production plant the company woul...,positive,0.999855
4,positive,According to the company 's updated strategy f...,positive,0.999766
5,positive,FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is ag...,positive,0.999806
6,positive,"For the last quarter of 2010 , Componenta 's n...",positive,0.999853
7,positive,"In the third quarter of 2010 , net sales incre...",positive,0.999842
8,positive,Operating profit rose to EUR 13.1 mn from EUR ...,positive,0.999813
9,positive,"Operating profit totalled EUR 21.1 mn , up fro...",positive,0.999816


## 1. Hugging Face: FinancialBERT-Sentiment-Analysis

https://huggingface.co/ahmedrachid/FinancialBERT-Sentiment-Analysis

Let's now explore integrating and testing FinancialBERT, a model designed specifically for sentiment analysis in the financial domain.

In [7]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline

model = BertForSequenceClassification.from_pretrained("ahmedrachid/FinancialBERT-Sentiment-Analysis",num_labels=3)
tokenizer = BertTokenizer.from_pretrained("ahmedrachid/FinancialBERT-Sentiment-Analysis")
hfmodel = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)


### Initialize VM dataset

In [8]:
# Load a test dataset with 100 rows only
vm_test_ds = vm.init_dataset(
    dataset=df.head(100),
    text_column="Sentence",
    target_column="Sentiment",
)

2023-09-08 16:59:24,152 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...


### Initialize VM model

When initializing a VM model, we pre-calculate predictions on the test dataset. This operation can take a long time for large datasets.

In [9]:
vm_model_1 = vm.init_model(
    hfmodel,
    test_ds=vm_test_ds,
)

In [11]:
# Some errors are expected since some classifier metrics are not applicable to all models
full_suite = vm.run_test_suite(
    "classifier_model_validation",
    dataset=vm_test_ds,
    model=vm_model_1,
)

2023-09-08 16:59:29,363 - ERROR(validmind.vm_models.test_plan): Failed to run test 'classifier_in_sample_performance': (MissingRequiredTestContextError) Model Training Dataset 'model.train_ds' is a required input and must be passed as a keyword argument to the test plan
2023-09-08 16:59:29,370 - ERROR(validmind.vm_models.test_plan): Failed to run test 'pfi': (MissingRequiredTestContextError) Model Training Dataset 'model.train_ds' is a required input and must be passed as a keyword argument to the test plan
2023-09-08 16:59:34,443 - ERROR(validmind.vm_models.test_plan): Failed to run test 'pr_curve': (SkipTestError) Precision Recall Curve is only supported for binary classification models
2023-09-08 16:59:39,030 - ERROR(validmind.vm_models.test_plan): Failed to run test 'roc_curve': (SkipTestError) ROC Curve is only supported for binary classification models
2023-09-08 16:59:39,030 - ERROR(validmind.vm_models.test_plan): Failed to run test 'psi': (MissingRequiredTestContextError) Model

## 2. Hugging Face: distilroberta-finetuned-financial-news-sentiment-analysis

https://huggingface.co/mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis

The distilroberta model was fine-tuned on the phrasebank dataset: https://huggingface.co/datasets/financial_phrasebank.

In [12]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")
model = AutoModelForSequenceClassification.from_pretrained("mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")
hfmodel = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)


### Initialize VM model

In [13]:
vm_model_2 = vm.init_model(
    hfmodel,
    test_ds=vm_test_ds,
)

In [14]:
full_suite = vm.run_test_suite(
    "classifier_model_validation",
    dataset=vm_test_ds,
    model=vm_model_2,
    models=[vm_model_1]

)

2023-09-08 16:59:44,632 - ERROR(validmind.vm_models.test_plan): Failed to run test 'classifier_in_sample_performance': (MissingRequiredTestContextError) Model Training Dataset 'model.train_ds' is a required input and must be passed as a keyword argument to the test plan
2023-09-08 16:59:44,639 - ERROR(validmind.vm_models.test_plan): Failed to run test 'pfi': (MissingRequiredTestContextError) Model Training Dataset 'model.train_ds' is a required input and must be passed as a keyword argument to the test plan
2023-09-08 16:59:46,802 - ERROR(validmind.vm_models.test_plan): Failed to run test 'pr_curve': (SkipTestError) Precision Recall Curve is only supported for binary classification models
2023-09-08 16:59:48,990 - ERROR(validmind.vm_models.test_plan): Failed to run test 'roc_curve': (SkipTestError) ROC Curve is only supported for binary classification models
2023-09-08 16:59:48,991 - ERROR(validmind.vm_models.test_plan): Failed to run test 'psi': (MissingRequiredTestContextError) Model

## 3. Hugging Face: financial-roberta-large-sentiment

https://huggingface.co/soleimanian/financial-roberta-large-sentiment

The financial-roberta-large model is another financial sentiment analysis model trained on large amounts of data including:

- Financial Statements
- Earnings Announcements
- Earnings Call Transcripts
- Corporate Social Responsibility (CSR) Reports
- Environmental, Social, and Governance (ESG) News
- Financial News
- Etc.

In [15]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("soleimanian/financial-roberta-large-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("soleimanian/financial-roberta-large-sentiment")
hfmodel = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)


In [16]:
vm_model_3 = vm.init_model(
    hfmodel,
    test_ds=vm_test_ds,
)

In [17]:
full_suite = vm.run_test_suite(
    "classifier_model_validation",
    dataset=vm_test_ds,
    model=vm_model_3,
    models=[vm_model_1, vm_model_2]
)

2023-09-08 17:00:08,942 - ERROR(validmind.vm_models.test_plan): Failed to run test 'classifier_in_sample_performance': (MissingRequiredTestContextError) Model Training Dataset 'model.train_ds' is a required input and must be passed as a keyword argument to the test plan
2023-09-08 17:00:08,954 - ERROR(validmind.vm_models.test_plan): Failed to run test 'pfi': (MissingRequiredTestContextError) Model Training Dataset 'model.train_ds' is a required input and must be passed as a keyword argument to the test plan
2023-09-08 17:00:23,020 - ERROR(validmind.vm_models.test_plan): Failed to run test 'pr_curve': (SkipTestError) Precision Recall Curve is only supported for binary classification models
2023-09-08 17:00:37,171 - ERROR(validmind.vm_models.test_plan): Failed to run test 'roc_curve': (SkipTestError) ROC Curve is only supported for binary classification models
2023-09-08 17:00:37,172 - ERROR(validmind.vm_models.test_plan): Failed to run test 'psi': (MissingRequiredTestContextError) Model